In [ ]:
import pandas as pd

# Sample DataFrame
data = {
    'desk': ['A', 'A', 'B', 'B', 'C', 'C'],
    'order_type': ['X', 'X', 'X', 'X', 'X', 'X'],
    'client': ['Client1', 'Client1', 'Client1', 'Client1', 'Client1', 'Client2'],
    'target': [0, 1, 0, 1, 1, 0]
}

df = pd.DataFrame(data)

# Step 1: Group by 'order_type' and 'client' to identify differences in delay outcomes across desks
grouped = df.groupby(['order_type', 'client']).filter(lambda x: x['target'].nunique() > 1)

# Step 2: Filter those records where there's at least one delay (1) and one no-delay (0) for the same 'order_type' and 'client'
mixed_outcome = grouped.groupby(['order_type', 'client', 'desk']).filter(lambda x: x['target'].nunique() > 0)

# Step 3: Identify which desks are responsible for delays
delay_desks = mixed_outcome[mixed_outcome['target'] == 1]

# Output the results
print("Desks responsible for delays on the same order type and client:")
print(delay_desks[['order_type', 'client', 'desk']].drop_duplicates())


In [ ]:
import pandas as pd

# Sample DataFrame
data = {
    'desk': ['A', 'A', 'B', 'B', 'C', 'C'],
    'order_type': ['X', 'X', 'X', 'X', 'X', 'X'],
    'client': ['Client1', 'Client1', 'Client1', 'Client1', 'Client1', 'Client2'],
    'target': [0, 1, 0, 1, 1, 0]
}

df = pd.DataFrame(data)

# Step 1: Group by 'order_type' and 'client' to identify differences in delay outcomes across desks
grouped = df.groupby(['order_type', 'client']).filter(lambda x: x['target'].nunique() > 1)

# Step 2: Separate records with delay (target == 1) and no-delay (target == 0)
delay_desks = grouped[grouped['target'] == 1].copy()
no_delay_desks = grouped[grouped['target'] == 0].copy()

# Step 3: Merge delayed desks with no-delay desks on 'order_type' and 'client'
comparison = pd.merge(delay_desks, no_delay_desks, on=['order_type', 'client'], suffixes=('_delayed', '_not_delayed'))

# Step 4: Output the comparison
print("Comparison of desks with and without delays for the same order type and client:")
print(comparison[['order_type', 'client', 'desk_delayed', 'desk_not_delayed']])
